In [1]:
import dash
import dash_html_components as html
import dash_core_components as dcc
import pandas as pd
from dash.dependencies import Output,Input
import plotly.express as px

In [2]:
df=pd.read_csv("https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_dash.csv")

In [3]:
df.head()

,Unnamed: 0,Flight Number,Launch Site,Mission Outcome,class,Payload Mass (kg),Booster Version,Booster Version Category
0,0,1,CCAFS LC-40,Success,0,0.0,F9 v1.0 B0003,v1.0
1,1,2,CCAFS LC-40,Success,0,0.0,F9 v1.0 B0004,v1.0
2,2,3,CCAFS LC-40,Success,0,525.0,F9 v1.0 B0005,v1.0
3,3,4,CCAFS LC-40,Success,0,500.0,F9 v1.0 B0006,v1.0
4,4,5,CCAFS LC-40,Success,0,677.0,F9 v1.0 B0007,v1.0


In [4]:
data=df[df['Launch Site']=='CCAFS LC-40']
data=data.groupby(['class']).size().reset_index()
data.columns=['success','count']
data

,success,count
0,0,19
1,1,7


In [5]:
df.isnull().sum()

Unnamed: 0                  0
Flight Number               0
Launch Site                 0
Mission Outcome             0
class                       0
Payload Mass (kg)           0
Booster Version             0
Booster Version Category    0
dtype: int64

In [6]:
df.drop('Unnamed: 0',axis=1,inplace=True)

In [7]:
Sites=df['Launch Site'].unique().tolist()
Sites

['CCAFS LC-40', 'VAFB SLC-4E', 'KSC LC-39A', 'CCAFS SLC-40']

In [8]:
min_value, max_value=0,10000

In [9]:
app=dash.Dash(__name__)

In [10]:
app.layout=html.Div([
    html.H1('SpaceX Launch Records dashboard',style={'textAlign':'center'}),
    html.Div([
        html.H2('select a launch site'),
        dcc.Dropdown(id='site-dropdown',
                options=[{'label': 'All Sites', 'value': 'ALL'},
                         {'label':'CCAFS LC-40','value':'CCAFS LC-40'},
                         {'label':'VAFB SLC-4E','value':'VAFB SLC-4E'},
                         {'label':'KSC LC-39A','value':'KSC LC-39A'},
                         {'label':'CCAFS SLC-40','value':'CCAFS SLC-40'}],
                         
                value='ALL',
                placeholder="place holder here",
                searchable=True,
                     style={'width':'80%', 'padding':'3px', 'font-size': '20px', 'text-align-last' : 'center'},
                ),
    ],style={'display':'flex'}),
    dcc.Graph(figure={},id='success-pie-chart'),
    html.Br(),
    html.Br(),
    html.Div([
        html.H2('Payload Range (kg)'),
        dcc.RangeSlider(id='payload-slider',
                min=0, max=10000, step=1000,
                marks={0: '0',
                       100: '100'},
                value=[min_value, max_value])
    ]),
    dcc.Graph(figure={},id='success-payload-scatter-chart')
])

In [11]:
# Function decorator to specify function input and output
@app.callback([Output(component_id='success-pie-chart', component_property='figure'),
              Output(component_id='success-payload-scatter-chart', component_property='figure')],
              [Input(component_id='site-dropdown', component_property='value'),
               Input(component_id="payload-slider", component_property="value")])
def get_graph(entered_site,payload_range):
    
    if entered_site == 'ALL':
        dataP=df[['class','Launch Site']].groupby(['Launch Site']).mean().reset_index()
        dataP.columns=['Launch Site','success']
        dataS=df[(df['Payload Mass (kg)']<payload_range[1])&(df['Payload Mass (kg)']>payload_range[0])]
        fig1 = px.pie(dataP, values='success', names='Launch Site', title='Total Success Launches By Site')
        fig2=px.scatter(dataS,x='Payload Mass (kg)',y='class',color='Booster Version Category',
                        title='Correlation Between PayloadMass And Success For All Sites')
        return fig1,fig2
    else:
        dataS=df[df['Launch Site']==entered_site]
        dataP=dataS.groupby(['class']).size().reset_index()
        dataP.columns=['success','count']
        dataS=dataS[(df['Payload Mass (kg)']<payload_range[1])&(df['Payload Mass (kg)']>payload_range[0])]
        title_pie='Total Success Launches For Site {}'.format(entered_site)
        title_scatter='Correlation Between PayloadMass And Success For Site {}'.format(entered_site)
        fig1=px.pie(dataP, values='count', names='success', title=title_pie)
        fig2=px.scatter(dataS,x='Payload Mass (kg)',y='class',color='Booster Version Category',title=title_scatter)
        return fig1,fig2


In [ ]:
if __name__ == '__main__':
    app.run_server()